#### Week 5 Assignment - The Battle of the Neighbourhoods

###### Shusanah Pillinger

##### 1. Background Introduction

I would like to go for a night out in London with a group of friends. We enjoy going to popular bars and pubs and tend to go to several venues on a night out. We live across London and will each travel to a neighbourhood by the underground train network - known as The Tube in London. The furthest we will travel is Zone 5 on the network. We will meet at a Tube station before proceeding to a bar or pub for drinks. After drinks we will probably head for food, so will head in a direction where restaurants are plentiful. We don't mind walking a bit further and can head home from a different Tube station if necessary. Which Tube station should we meet at for a successful night out?

##### 2. Problem Definition

Problem 1: Which Tube stations are nearest the highest rated bars and pubs on Foursquare?

Problem 2: Of the Tube stations with at least 5 nearby bars or pubs, what are 10 most common venue categories nearby?

Problem 3: Which group of Tube stations offer the best choice for a meal after drinks?

Culminating in determining the answer to "Which Tube station should we meet at for a successful night out?"

##### 3. Data Requirements

1) Require Tube station locations so that latitude and longitude coordinates and which Zone the stations are in can be determined. Information about London Underground stations can be found on Wikipedia: https://en.wikipedia.org/wiki/List_of_London_Underground_stations

2) Venue data for a limited number of venues within 500m of Tube stations within Zones 1 to 5 is required so that specifically bar and pub data can be analysed further. This will involve calls using the Foursquare API to search nearby venues with the query “Bar”. 

3) For each Tube station, require a summary of how many pubs and bars, the number of reviews and the average rating for pubs and bars in the surrounding area, so that those Tube stations requiring further exploration can be determined. This will require data manipulation of the contents of a dataframe.

4) Then for the selected Tube stations with at least 5 bars or pubs, require details of the 10 most common venue categories within 1000m of each Tube station, so that data can be grouped according to venue type. This venue data and categorisation will also be collated from the Foursquare City Guide using their explore API.

5) Require clustering of the Tube stations by the similarity of venue category nearby, so an area of London with plentiful restaurant choice can be determined.

##### 4. Methodology

##### 4.1 Libraries required for Data Analysis

In [1]:
import io
import pandas as pd
import requests
import numpy as np
!pip install lxml 
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
# import k-means from clustering stage
from sklearn.cluster import KMeans
#!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

     |████████████████████████████████| 5.8MB 32.8MB/s eta 0:00:01
Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.2 MB

The following NEW packages will be INSTALLED:

    geographiclib: 1.50-py_0         conda-forg

##### 4.2. Retrieval of London Tube station data

In [2]:
wikipedia_link='https://en.wikipedia.org/wiki/List_of_London_Underground_stations'
wikipedia_page_html = requests.get(wikipedia_link)
wikipedia_page_text = wikipedia_page_html.text
df_wiki_data = pd.read_html(wikipedia_page_text, header=0)[0]
df_wiki_data.head()

,Station,Photograph,Line(s)[*],Local authority,Zone(s)[†],Opened[4],Main lineopened,Other name(s)[note 2],Usage[5]
0,Acton Town,NaN,DistrictPiccadilly,Ealing,3,1 July 1879,NaN,Mill Hill Park: 1879–1910,6.04
1,Aldgate,NaN,Metropolitan[a]Circle,City of London,1,18 November 1876,NaN,NaN,8.85
2,Aldgate East,NaN,Hammersmith & City[d]District,Tower Hamlets,1,6 October 1884resited 31 October 1938,NaN,Commercial Road: Proposed before opening,14.00
3,Alperton,NaN,Piccadilly[h],Brent,4,28 June 1903,NaN,Perivale-Alperton: 1903–10,3.05
4,Amersham,NaN,Metropolitan,Chiltern,9,1 September 1892,NaN,Amersham: 1892–1922Amersham & Chesham Bois: 19...,2.32


##### 4.3. Process data

In [3]:
# Remove unnecessary columns and rename Zone column
df_tube_stations = df_wiki_data[['Station','Zone(s)[†]']]
df_tube_stations = df_tube_stations.rename(columns = {'Zone(s)[†]':'Zone'})
df_tube_stations.head()

,Station,Zone
0,Acton Town,3
1,Aldgate,1
2,Aldgate East,1
3,Alperton,4
4,Amersham,9


In [4]:
# Size of dataframe
df_tube_stations.shape

(270, 2)

In [5]:
# Count of stations by Zone
df_tube_stations.groupby(['Zone']).size().reset_index(name='Count')

,Zone,Count
0,1,60
1,1 & 2,4
2,2,57
3,2 & 3,14
4,3,35
5,3 & 4,6
6,4,44
7,5,22
8,5 & 6,1
9,6,19


In [6]:
# Remove Tube stations which are outside of Zone 5. In this case also include station which is in Zone 5 & 6
df_tube_stations = df_tube_stations.drop(df_tube_stations[(df_tube_stations['Zone'] == '5 & 6') |
                                                          (df_tube_stations['Zone'] == '6') |
                                                          (df_tube_stations['Zone'] == '6 & 7') |
                                                          (df_tube_stations['Zone'] == '7') |
                                                          (df_tube_stations['Zone'] == '8') |
                                                          (df_tube_stations['Zone'] == '9')].index)
df_tube_stations.reset_index(drop=True, inplace=True)
df_tube_stations.head()

,Station,Zone
0,Acton Town,3
1,Aldgate,1
2,Aldgate East,1
3,Alperton,4
4,Angel,1


In [7]:
# Size of dataframe
df_tube_stations.shape

(242, 2)

In [8]:
# Count of stations by Zone
df_tube_stations.groupby(['Zone']).size().reset_index(name='Count')

,Zone,Count
0,1,60
1,1 & 2,4
2,2,57
3,2 & 3,14
4,3,35
5,3 & 4,6
6,4,44
7,5,22


##### 4.4. Determine latitude and longitude for each station

In [9]:
# Retrieve latitude and longitude from source on Open Street Map
osm_link='https://wiki.openstreetmap.org/wiki/List_of_London_Underground_stations#Tube_Stations_List'
osm_page_html = requests.get(osm_link)
osm_page_text = osm_page_html.text
df_osm_data = pd.read_html(osm_page_text, header=0)[0]
df_osm_data.head()

,Name,Latitude,Longitude,Platform / Entrance,Collected By,Collected On,Line,Step free
0,Acton Town,51.502500,-0.278126,Platform,User:Gagravarr,24/11/06,"District, Piccadilly",NaN
1,Acton Central,51.50883531,-0.263033174,Entrance,User:Firefishy,08/05/2007,London Overground,NaN
2,Acton Central,51.50856013,-0.262879534,Platform,User:Firefishy,08/05/2007,London Overground,NaN
3,Aldgate,51.51394,-0.07537,Aldgate High Street entrance,User:Morwen,28/4/2007,Metropolitan,No
4,Aldgate East,51.51514,-0.07178,Entrance,User:Parsingphase,(2006),"District, Hammersmith & City",NaN


In [11]:
# Remove unnecessary columns and rename Name column
df_osm_stations = df_osm_data[['Name','Latitude','Longitude']]
df_osm_stations.rename(columns = {'Name':'Station'}, inplace = True)
df_osm_stations.head()

,Station,Latitude,Longitude
0,Acton Town,51.502500,-0.278126
1,Acton Central,51.50883531,-0.263033174
2,Acton Central,51.50856013,-0.262879534
3,Aldgate,51.51394,-0.07537
4,Aldgate East,51.51514,-0.07178


In [12]:
# Join dataframes to get Station, Zone, Latitude and Longitude
df_tube_stations = df_tube_stations.join(df_osm_stations.set_index('Station'), on='Station')
df_tube_stations

,Station,Zone,Latitude,Longitude
0,Acton Town,3,51.502500,-0.278126
1,Aldgate,1,51.51394,-0.07537
2,Aldgate East,1,51.51514,-0.07178
3,Alperton,4,51.54097,-0.30061
4,Angel,1,51.53253,-0.10579
...,...,...,...,...
237,Wimbledon Park,3,51.43391,-0.19864
238,Wood Green,3,51.59709,-0.10939
239,Wood Lane,2,NaN,NaN
240,Woodford,4,51.60582,+0.03328


In [13]:
# Check for duplicates of station name, except first occurence
df_duplicate_rows = df_tube_stations[df_tube_stations.duplicated('Station')]
print(df_duplicate_rows)


                  Station   Zone       Latitude       Longitude
8            Baker Street      1      51.522236       -0.157080
64           Edgware Road      1            NaN             NaN
70             Farringdon      1      51.520086       -0.104789
87            Hammersmith      2            NaN             NaN
144              Northolt      5      51.548467        -0.36968
153            Paddington      1  51.5151846554  -0.17553880792
170              Richmond      4     51.4632474    -0.300618047
198             Stratford  2 & 3      51.541146       -0.002675
205  Tottenham Court Road      1    51.51672074     -0.13093911
227        West Hampstead      2       51.54657        -0.19055
227        West Hampstead      2    51.54755518    -0.190999685
227        West Hampstead      2       51.54716        -0.19275
235    Willesden Junction  2 & 3    51.53032031    -0.229378995


In [14]:
# Drop duplicates
df_tube_stations = df_tube_stations.drop_duplicates(subset='Station',keep='first')
df_tube_stations.reset_index(drop=True, inplace=True)
df_tube_stations.head()

,Station,Zone,Latitude,Longitude
0,Acton Town,3,51.502500,-0.278126
1,Aldgate,1,51.51394,-0.07537
2,Aldgate East,1,51.51514,-0.07178
3,Alperton,4,51.54097,-0.30061
4,Angel,1,51.53253,-0.10579


In [15]:
# Size of dataframe
df_tube_stations.shape

(239, 4)

In [ ]:
# Replace non-numeric values for Latitude or Longitude with NaN
df_tube_stations['Latitude'] = pd.to_numeric(df_tube_stations['Latitude'], errors='coerce')
df_tube_stations['Longitude'] = pd.to_numeric(df_tube_stations['Longitude'], errors='coerce')
df_tube_stations.head()

In [18]:
# How many rows have NaN data?
df_tube_stations[df_tube_stations.isnull().any(axis=1)]

,Station,Zone,Latitude,Longitude
34,Canada Water,2,NaN,NaN
36,Canning Town,2 & 3,NaN,NaN
63,Edgware Road,1,NaN,NaN
85,Hammersmith,2,NaN,NaN
112,King's Cross St Pancras,1,NaN,NaN
172,St. James's Park,1,NaN,NaN
173,St. John's Wood,2,NaN,NaN
174,St. Paul's,1,NaN,NaN
236,Wood Lane,2,NaN,NaN


In [20]:
# Manually input locations for NaN
df_tube_stations.loc[df_tube_stations['Station'] == 'Canada Water', ['Latitude']] = 51.497850 
df_tube_stations.loc[df_tube_stations['Station'] == 'Canada Water', ['Longitude']] = -0.049629
df_tube_stations.loc[df_tube_stations['Station'] == 'Canning Town', ['Latitude']] = 51.514455 
df_tube_stations.loc[df_tube_stations['Station'] == 'Canning Town', ['Longitude']] = 0.008851
df_tube_stations.loc[df_tube_stations['Station'] == 'Edgware Road', ['Latitude']] = 51.520170
df_tube_stations.loc[df_tube_stations['Station'] == 'Edgware Road', ['Longitude']] = -0.170226
df_tube_stations.loc[df_tube_stations['Station'] == 'Hammersmith', ['Latitude']] = 51.493491 
df_tube_stations.loc[df_tube_stations['Station'] == 'Hammersmith', ['Longitude']] = -0.224710
df_tube_stations.loc[df_tube_stations['Station'] == "King's Cross St Pancras", ['Latitude']] = 51.531604
df_tube_stations.loc[df_tube_stations['Station'] == "King's Cross St Pancras", ['Longitude']] =  -0.125399
df_tube_stations.loc[df_tube_stations['Station'] == "St. James's Park", ['Latitude']] = 51.499698 
df_tube_stations.loc[df_tube_stations['Station'] == "St. James's Park", ['Longitude']] = -0.133255
df_tube_stations.loc[df_tube_stations['Station'] == "St. John's Wood", ['Latitude']] = 51.534541 
df_tube_stations.loc[df_tube_stations['Station'] == "St. John's Wood", ['Longitude']] = -0.173978
df_tube_stations.loc[df_tube_stations['Station'] == "St. Paul's", ['Latitude']] = 51.514886
df_tube_stations.loc[df_tube_stations['Station'] == "St. Paul's", ['Longitude']] = -0.097250
df_tube_stations.loc[df_tube_stations['Station'] == 'Wood Lane', ['Latitude']] = 51.510010 
df_tube_stations.loc[df_tube_stations['Station'] == 'Wood Lane', ['Longitude']] = -0.224644

In [24]:
# Export tube stations to csv for use without having to regenerate all the data
df_tube_stations.to_csv("tube_stations.csv", encoding='utf-8', index=False)

In [21]:
# Debug: import tube stations from CSV instead of regenerating all the data
df_tube_stations = pd.read_csv("tube_stations.csv")

##### 4.5. Plot a map of London to show Tube stations

In [22]:
# Get co-ordinates for a map of London
address = 'London'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geographical coordinates of London are {}, {}.'.format(latitude, longitude))

The geographical coordinates of London are 51.5073219, -0.1276474.


In [23]:
# Draw map with Tube stations
map_london = folium.Map(location=[latitude, longitude], tiles='Stamen Toner', zoom_start=11)
for lat, lng, station, zone in zip(df_tube_stations['Latitude'], df_tube_stations['Longitude'], df_tube_stations['Station'],df_tube_stations['Zone']):
    popup = folium.Popup(station, parse_html=True)
    if (zone == '1' or zone == '1 & 2'):
        color = 'red',
        fill_color = '#F08080'
    elif (zone == '2' or zone == '2 & 3'):
        color = 'orange',
        fill_color = '#FFE4B5'
    elif (zone == '3' or zone == '3 & 4'):
        color = 'yellow',
        fill_color = '#FFFFE0'
    elif (zone == '4' or zone == '4 & 5'):
        color = 'green',
        fill_color = '#98FB98'
    elif zone == '5':
        color = 'blue',
        fill_color = '#3186cc'
    else:
        color = 'black'
    folium.CircleMarker(
        [lat, lng],
        radius = 5,
        popup = popup,
        color = color,
        fill = True,
        fill_color = fill_color,
        fill_opacity = 0.7,
        parse_html = False).add_to(map_london)  
    
map_london

##### 4.6. Restrict dataset to just Zone 1 (due to number limits on Foursquare calls)

In [24]:
is_zone_1 = df_tube_stations['Zone']=='1'
df_tube_stations_zone_1 = df_tube_stations[is_zone_1]
df_tube_stations_zone_1.head()

,Station,Zone,Latitude,Longitude
1,Aldgate,1,51.513940,-0.075370
2,Aldgate East,1,51.515140,-0.071780
4,Angel,1,51.532530,-0.105790
8,Baker Street,1,51.522650,-0.157040
10,Bank,1,51.513405,-0.089058


In [29]:
# Export Zone 1 tube stations to csv for use without having to regenerate all the data
df_tube_stations_zone_1.to_csv("tube_stations_zone_1.csv", encoding='utf-8', index=False)

In [25]:
# Debug: import Zone 1 tube stations from CSV instead of regenerating all the data
df_tube_stations_zone_1 = pd.read_csv("tube_stations_zone_1.csv")

In [31]:
# Debug: test set with only one record
is_Aldgate = df_tube_stations['Station']=='Aldgate'
df_tube_stations_test = df_tube_stations[is_Aldgate]
df_tube_stations_test.head()

,Station,Zone,Latitude,Longitude
1,Aldgate,1,51.51394,-0.07537


##### 4.7. Retrieve Foursquare venue data for required Tube stations

In [26]:
# Setup Foursquare credentials
CLIENT_ID = 'XIYI2B11PNWFYNEDFPUGBGTAZHI3N0052CVGSW4EL53OM32M' # your Foursquare ID
CLIENT_SECRET = '0N4JOTYWIEYUXCMBYGR3SJ4YJKUCXGGXU4UEWYU0WJYLKVNV' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version


print('Your credentials:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentials:
CLIENT_ID: XIYI2B11PNWFYNEDFPUGBGTAZHI3N0052CVGSW4EL53OM32M
CLIENT_SECRET:0N4JOTYWIEYUXCMBYGR3SJ4YJKUCXGGXU4UEWYU0WJYLKVNV


In [27]:
# Function to find bars and pubs near Tube stations
def getNearbyBars(stations, latitudes, longitudes, zones, radius=500):
    
    bars_list=[]
    for station, lat, lng, zone in zip(stations, latitudes, longitudes, zones):
        print(station)
            
        # category for bar (includes pub) -> 4bf58dd8d48988d116941735
        # categoryId = '4bf58dd8d48988d116941735'  
        search_query = 'Bar'
        LIMIT = 5 # Ordinarily I would search for more, but quota limits are restrictive
        
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&query={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT,
            search_query)
            
        # make the GET request
        results = requests.get(url).json()['response']['venues']
        
        # The following can be used to check for quota limits being exceeded
        # print(results) 
        #print(results['response'])
        #print(results['response'].keys())
        
        # return only relevant information for each nearby venue
        bars_list.append([(
            station, 
            lat, 
            lng,
            zone,
            v['id'],
            v['name'], 
            v['location']['lat'], 
            v['location']['lng']) for v in results])

    df_nearby_bars = pd.DataFrame([item for bar_list in bars_list for item in bar_list])
    df_nearby_bars.columns = ['Station', 
                  'Station Latitude', 
                  'Station Longitude',
                  'Station Zone',
                  'Bar Id',
                  'Bar Name', 
                  'Bar Latitude', 
                  'Bar Longitude']
    
    return(df_nearby_bars)

In [34]:
# Test - call function to get nearby bars for single Tube station
#df_london_bars = getNearbyBars(stations=df_tube_stations_test['Station'],
#                               latitudes=df_tube_stations_test['Latitude'],
#                               longitudes=df_tube_stations_test['Longitude'],
#                               zones=df_tube_stations_test['Zone'])
# Call function to get nearby bars for each Tube stations
df_london_bars_zone_1 = getNearbyBars(stations=df_tube_stations_zone_1['Station'],
                               latitudes=df_tube_stations_zone_1['Latitude'],
                               longitudes=df_tube_stations_zone_1['Longitude'],
                               zones=df_tube_stations_zone_1['Zone'])
df_london_bars_zone_1.head()

Aldgate
Aldgate East
Angel
Baker Street
Bank
Barbican
Bayswater
Blackfriars
Bond Street
Borough
Cannon Street
Chancery Lane
Charing Cross
Covent Garden
Edgware Road
Embankment
Euston
Euston Square
Farringdon
Gloucester Road
Goodge Street
Great Portland Street
Green Park
High Street Kensington
Holborn
Hyde Park Corner
King's Cross St Pancras
Knightsbridge
Lambeth North
Lancaster Gate
Leicester Square
Liverpool Street
London Bridge
Mansion House
Marble Arch
Marylebone
Monument
Moorgate
Old Street
Oxford Circus
Paddington
Piccadilly Circus
Pimlico
Queensway
Regent's Park
Russell Square
St. James's Park
St. Paul's
Sloane Square
South Kensington
Southwark
Temple
Tottenham Court Road
Tower Hill
Victoria
Warren Street
Waterloo
Westminster


,Station,Station Latitude,Station Longitude,Station Zone,Bar Id,Bar Name,Bar Latitude,Bar Longitude
0,Aldgate,51.51394,-0.07537,1,4c8a86e09ef0224b8db55b7b,All Bar One,51.519910,-0.079129
1,Aldgate,51.51394,-0.07537,1,4b7d84d6f964a520f3c32fe3,All Bar One,51.515802,-0.079613
2,Aldgate,51.51394,-0.07537,1,4ac518c3f964a5204fa420e3,T Bar,51.514445,-0.077505
3,Aldgate,51.51394,-0.07537,1,4ad0684ff964a520fad720e3,Isis Lounge Bar,51.510813,-0.077102
4,Aldgate,51.51394,-0.07537,1,4e6bed5c45dd435bc0d2a592,Grange Tower Bar,51.511667,-0.071745


In [35]:
# Show dataframe results
df_london_bars_zone_1

,Station,Station Latitude,Station Longitude,Station Zone,Bar Id,Bar Name,Bar Latitude,Bar Longitude
0,Aldgate,51.51394,-0.07537,1,4c8a86e09ef0224b8db55b7b,All Bar One,51.519910,-0.079129
1,Aldgate,51.51394,-0.07537,1,4b7d84d6f964a520f3c32fe3,All Bar One,51.515802,-0.079613
2,Aldgate,51.51394,-0.07537,1,4ac518c3f964a5204fa420e3,T Bar,51.514445,-0.077505
3,Aldgate,51.51394,-0.07537,1,4ad0684ff964a520fad720e3,Isis Lounge Bar,51.510813,-0.077102
4,Aldgate,51.51394,-0.07537,1,4e6bed5c45dd435bc0d2a592,Grange Tower Bar,51.511667,-0.071745
...,...,...,...,...,...,...,...,...
285,Westminster,51.50121,-0.12489,1,4c506074371520a150fd38c1,Strangers' Bar,51.499564,-0.123919
286,Westminster,51.50121,-0.12489,1,4b9b933ff964a520a21036e3,Neds Noodle Bar,51.502119,-0.117971
287,Westminster,51.50121,-0.12489,1,4e70ee5bc65b340cdea0ebb8,"Peers Bar, House Of Lords",51.501998,-0.126064
288,Westminster,51.50121,-0.12489,1,4f60dcbfe5e8d4902df2c210,Gillray's Steakhouse & Bar,51.501824,-0.119821


In [36]:
# Size of dataframe
df_london_bars_zone_1.shape

(290, 8)

In [38]:
# Export london bars to csv for use without having to regenerate all the data
df_london_bars_zone_1.to_csv("london_bars_zone_1.csv", encoding='utf-8', index=False)

In [28]:
# Debug: import London bars from CSV instead of regenerating all the data
df_london_bars_zone_1 = pd.read_csv("london_bars_zone_1.csv")

In [29]:
# Count of bars by Tube Station
pd.set_option("display.max_rows", 300)
df_london_bars_zone_1.groupby(['Station']).size().reset_index(name='Count')

,Station,Count
0,Aldgate,5
1,Aldgate East,5
2,Angel,5
3,Baker Street,5
4,Bank,5
5,Barbican,5
6,Bayswater,5
7,Blackfriars,5
8,Bond Street,5
9,Borough,5


##### 4.8. Enhance Foursquare venue data with Ratings and Tip Counts

In [36]:
# Add extra columns to dataframe to store rating and tip count, set initially to null
df_london_bars_zone_1["Rating"] = 0
df_london_bars_zone_1["Tip Count"] = 0
df_london_bars_zone_1.head()
df_london_bars_zone_1.dtypes

Station               object
Station Latitude     float64
Station Longitude    float64
Station Zone           int64
Bar Id                object
Bar Name              object
Bar Latitude         float64
Bar Longitude        float64
Rating                 int64
Tip Count              int64
dtype: object

In [37]:
# Function to retrieve rating and tip count
def getRatingAndTipCount(bar_ids):
    
    for bar_id in (bar_ids):
        print(bar_id)
            
        venue_id = bar_id 
        url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(
            venue_id, 
            CLIENT_ID,  
            CLIENT_SECRET, 
            VERSION)

        # make the GET request
        result = requests.get(url).json()
        
        # inspect rating
        try:
            rating = result['response']['venue']['rating']
        except:
            print('No Rating')
            rating = 0
        # update London Bars dataframe with rating
        df_london_bars_zone_1.loc[df_london_bars_zone_1['Bar Id'] == venue_id, ['Rating']] = rating

        # inspect tip count
        try:
            tip_count = result['response']['venue']['tips']['count']
        except:
            tip_count = 0
        #update London Bars dataframe with tip count
        df_london_bars_zone_1.loc[df_london_bars_zone_1['Bar Id'] == venue_id, ['Tip Count']] = tip_count
    
    return()

In [ ]:
# Call function to get ratings and tip counts for each venue
getRatingAndTipCount(bar_ids=df_london_bars_zone_1['Bar Id'])
df_london_bars_zone_1.head()

In [47]:
# Export london bars with ratings and tip counts to csv for use without having to regenerate all the data
df_london_bars_zone_1.to_csv("london_bars_zone_1_rated.csv", encoding='utf-8', index=False)

In [30]:
# Debug: import London bars with ratings and tip counts from CSV instead of regenerating all the data
df_london_bars_zone_1 = pd.read_csv("london_bars_zone_1_rated.csv")

In [39]:
# In order to get mean of ratings not including zero ratings, sum and then divide by number of non-zero ratings
# First find number of non-zero ratings
df_london_bars_zone_1_with_rating = df_london_bars_zone_1[df_london_bars_zone_1['Rating'] > 0]
df_number_of_non_zero_ratings = df_london_bars_zone_1_with_rating.groupby('Station')['Rating'].size().reset_index(name='Count')
df_number_of_non_zero_ratings.head()

,Station,Count
0,Aldgate,3
1,Angel,3
2,Baker Street,4
3,Bank,3
4,Barbican,5


In [40]:
# Combine London bar ratings with count of non-zero ratings
df_london_bars_zone_1 = df_london_bars_zone_1.join(df_number_of_non_zero_ratings.set_index('Station'), on='Station')
df_london_bars_zone_1.head()

,Station,Station Latitude,Station Longitude,Station Zone,Bar Id,Bar Name,Bar Latitude,Bar Longitude,Rating,Tip Count,Count
0,Aldgate,51.51394,-0.07537,1,4c8a86e09ef0224b8db55b7b,All Bar One,51.519910,-0.079129,5.6,6,3.0
1,Aldgate,51.51394,-0.07537,1,4b7d84d6f964a520f3c32fe3,All Bar One,51.515802,-0.079613,5.9,7,3.0
2,Aldgate,51.51394,-0.07537,1,4ac518c3f964a5204fa420e3,T Bar,51.514445,-0.077505,0.0,3,3.0
3,Aldgate,51.51394,-0.07537,1,4ad0684ff964a520fad720e3,Isis Lounge Bar,51.510813,-0.077102,5.9,2,3.0
4,Aldgate,51.51394,-0.07537,1,4e6bed5c45dd435bc0d2a592,Grange Tower Bar,51.511667,-0.071745,0.0,0,3.0


In [41]:
# Summarise bars near Tube stations with total rating and total number of tip counts
df_london_bar_totals_zone_1 = (df_london_bars_zone_1.groupby(by=['Station', 
                                                           'Station Latitude',
                                                           'Station Longitude',
                                                           'Station Zone',
                                                           'Count'],as_index=False)
       .agg({'Rating':'sum','Tip Count':'sum'})
       .rename(columns={'Rating':'Total Rating', 'Tip Count':'Total Tip Count'}))
df_london_bar_totals_zone_1.head()

,Station,Station Latitude,Station Longitude,Station Zone,Count,Total Rating,Total Tip Count
0,Aldgate,51.513940,-0.075370,1,3.0,17.4,18
1,Angel,51.532530,-0.105790,1,3.0,16.1,8
2,Baker Street,51.522650,-0.157040,1,4.0,29.7,166
3,Bank,51.513405,-0.089058,1,3.0,20.9,9
4,Barbican,51.520865,-0.097758,1,5.0,33.1,44


In [42]:
# Calculate the average rating as total rating divided by number of non-zero ratings
df_london_bar_totals_zone_1['Average Rating'] = df_london_bar_totals_zone_1['Total Rating']/df_london_bar_totals_zone_1['Count']
df_london_bar_totals_zone_1.head()

,Station,Station Latitude,Station Longitude,Station Zone,Count,Total Rating,Total Tip Count,Average Rating
0,Aldgate,51.513940,-0.075370,1,3.0,17.4,18,5.800000
1,Angel,51.532530,-0.105790,1,3.0,16.1,8,5.366667
2,Baker Street,51.522650,-0.157040,1,4.0,29.7,166,7.425000
3,Bank,51.513405,-0.089058,1,3.0,20.9,9,6.966667
4,Barbican,51.520865,-0.097758,1,5.0,33.1,44,6.620000


In [43]:
# Summarise by removing unnecessary columns
del df_london_bar_totals_zone_1['Count']
del df_london_bar_totals_zone_1['Total Rating']
df_london_bar_summary_zone_1 = df_london_bar_totals_zone_1.rename(columns = {'Station Latitude':'Latitude','Station Longitude':'Longitude','Station Zone':'Zone'})
df_london_bar_summary_zone_1.head()

,Station,Latitude,Longitude,Zone,Total Tip Count,Average Rating
0,Aldgate,51.513940,-0.075370,1,18,5.800000
1,Angel,51.532530,-0.105790,1,8,5.366667
2,Baker Street,51.522650,-0.157040,1,166,7.425000
3,Bank,51.513405,-0.089058,1,9,6.966667
4,Barbican,51.520865,-0.097758,1,44,6.620000


In [46]:
# Export london bar summary to csv for use without having to regenerate all the data
df_london_bar_summary_zone_1.to_csv("london_bars_summary_zone_1.csv", encoding='utf-8', index=False)

In [31]:
# Debug: import London bar summary from CSV instead of regenerating all the data
df_london_bar_summary_zone_1 = pd.read_csv("london_bars_summary_zone_1.csv")

##### 4.9. Plot a map of London to show Tube stations in Zone 1 by Average Rating of local bars

In [32]:
# Draw map with Tube stations in Zone 1 and Average Rating
map_london_bar_ratings = folium.Map(location=[latitude, longitude], tiles='Stamen Toner', zoom_start=13)
for lat, lng, station, tip, rating in zip(df_london_bar_summary_zone_1['Latitude'], 
                                   df_london_bar_summary_zone_1['Longitude'], 
                                   df_london_bar_summary_zone_1['Station'],
                                   df_london_bar_summary_zone_1['Total Tip Count'],
                                   df_london_bar_summary_zone_1['Average Rating']):
    popup = folium.Popup(station + ": Average Rating " + str(round(rating,1)) + ", Total Tip Count " + str(tip), parse_html=True)
    if (rating <= 6):
        color = 'red',
        fill_color = '#F08080'
    elif (rating > 6 and rating <= 7):
        color = 'orange',
        fill_color = '#FFE4B5'
    elif (rating > 7 and rating <= 8):
        color = 'yellow',
        fill_color = '#FFFFE0'
    elif (rating > 8 and rating <= 9):
        color = 'green',
        fill_color = '#98FB98'
    elif (rating > 9 and rating <= 10):
        color = 'blue',
        fill_color = '#3186cc'
    else:
        color = 'black'
    folium.CircleMarker(
        [lat, lng],
        radius = 5,
        popup = popup,
        color = color,
        fill = True,
        fill_color = fill_color,
        fill_opacity = 0.7,
        parse_html = False).add_to(map_london_bar_ratings)  
    
map_london_bar_ratings

##### 4.10. Explore other Foursquare venue data around the top rated bar areas

In [33]:
# Get a subset of Tube stations with the highest ratings to be further investigated for their proximity to other venue categories
# e.g. green, over 8 and yellow, over 7 
df_highest_rated = df_london_bar_summary_zone_1.drop(df_london_bar_summary_zone_1[(df_london_bar_summary_zone_1['Average Rating'] <= 7)].index)
df_highest_rated.reset_index(drop=True, inplace=True)
df_highest_rated

,Station,Latitude,Longitude,Zone,Total Tip Count,Average Rating
0,Baker Street,51.522650,-0.157040,1,166,7.425000
1,Blackfriars,51.511440,-0.104191,1,144,7.400000
2,Bond Street,51.514610,-0.148970,1,132,7.540000
3,Borough,51.500950,-0.094460,1,22,7.700000
4,Chancery Lane,51.518360,-0.111150,1,92,7.766667
5,Charing Cross,51.507108,-0.122963,1,465,8.100000
6,Covent Garden,51.513080,-0.124270,1,279,7.400000
7,Edgware Road,51.520170,-0.170226,1,16,7.200000
8,Embankment,51.507170,-0.121950,1,453,7.880000
9,Great Portland Street,51.523910,-0.143970,1,112,7.120000


In [47]:
# Function to retrieve recommended venue information for venues near Tube stations
def getNearbyVenues(stations, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    LIMIT = 50 # Higher that Search to get a bigger picture of surrounding area
    
    for station, lat, lng in zip(stations, latitudes, longitudes):
        print(station)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            station, 
            lat, 
            lng, 
            v['venue']['id'],
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    df_nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    df_nearby_venues.columns = ['Station', 
                  'Station Latitude', 
                  'Station Longitude',
                  'Venue Id',
                  'Venue Name', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(df_nearby_venues)

In [100]:
# Call function to get nearby venues for each Tube stations
df_london_venues = getNearbyVenues(stations=df_highest_rated['Station'],
                                   latitudes=df_highest_rated['Latitude'],
                                   longitudes=df_highest_rated['Longitude']
                                  )
df_london_venues.head()

Baker Street
Blackfriars
Bond Street
Borough
Chancery Lane
Charing Cross
Covent Garden
Edgware Road
Embankment
Great Portland Street
Green Park
High Street Kensington
Hyde Park Corner
Knightsbridge
Leicester Square
London Bridge
Mansion House
Marylebone
Monument
Old Street
Oxford Circus
Piccadilly Circus
Queensway
Regent's Park
Sloane Square
Southwark
Temple
Tottenham Court Road


,Station,Station Latitude,Station Longitude,Venue Id,Venue Name,Venue Latitude,Venue Longitude,Venue Category
0,Baker Street,51.52265,-0.15704,5964bee2a2c00b527a7e3e18,JOE & THE JUICE,51.520907,-0.157127,Juice Bar
1,Baker Street,51.52265,-0.15704,4f159852a17c8a14455c9580,Pret A Manger,51.522986,-0.158039,Sandwich Place
2,Baker Street,51.52265,-0.15704,4f8c0e4fe4b0e5ed76ce5083,Madame Tussauds 4D,51.522817,-0.154881,Movie Theater
3,Baker Street,51.52265,-0.15704,4ac518cef964a520fca520e3,Madame Tussauds,51.523017,-0.154361,Museum
4,Baker Street,51.52265,-0.15704,58440cd79850a03d83c5ac3c,Honest Burgers,51.520474,-0.155920,Burger Joint


In [101]:
# Size of dataframe
df_london_venues.shape

(1400, 8)

In [122]:
# Export london venues to csv for use without having to regenerate all the data
df_london_venues.to_csv("london_venues.csv", encoding='utf-8', index=False)

In [34]:
# Debug: import London venues from CSV instead of regenerating all the data
df_london_venues = pd.read_csv("london_venues.csv")

In [49]:
# Count of venues by category
df_london_venues.groupby(['Venue Category']).size().reset_index(name='Count')

,Venue Category,Count
0,American Restaurant,7
1,Argentinian Restaurant,7
2,Art Gallery,18
3,Art Museum,12
4,Arts & Crafts Store,2
5,Asian Restaurant,8
6,Austrian Restaurant,3
7,BBQ Joint,1
8,Bakery,22
9,Bar,13


##### 4.11. Determine most common venue categories around Tube stations of interest

In [36]:
# one hot encoding
df_london_onehot = pd.get_dummies(df_london_venues[['Venue Category']], prefix="", prefix_sep="")

# add Station column back to dataframe
df_london_onehot['Station'] = df_london_venues['Station'] 

# move Station column to the first column
fixed_columns = [df_london_onehot.columns[-1]] + list(df_london_onehot.columns[:-1])
df_london_onehot = df_london_onehot[fixed_columns]

df_london_onehot.head()

,Station,American Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Austrian Restaurant,BBQ Joint,Bakery,...,Udon Restaurant,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Watch Shop,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Baker Street,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Baker Street,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Baker Street,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Baker Street,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Baker Street,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [37]:
# Size of dataframe
df_london_onehot.shape

(1400, 199)

In [38]:
# Group rows by Station and by taking the mean of the frequency of occurrence of each category
df_london_grouped = df_london_onehot.groupby('Station').mean().reset_index()
df_london_grouped

,Station,American Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Austrian Restaurant,BBQ Joint,Bakery,...,Udon Restaurant,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Watch Shop,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Baker Street,0.00,0.00,0.00,0.00,0.00,0.00,0.02,0.00,0.02,...,0.00,0.00,0.00,0.00,0.00,0.00,0.02,0.00,0.00,0.02
1,Blackfriars,0.02,0.00,0.00,0.06,0.00,0.00,0.00,0.00,0.02,...,0.00,0.00,0.00,0.02,0.00,0.00,0.02,0.00,0.00,0.00
2,Bond Street,0.00,0.02,0.06,0.00,0.00,0.00,0.00,0.00,0.02,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.02,0.00,0.00
3,Borough,0.02,0.02,0.00,0.00,0.00,0.02,0.00,0.00,0.00,...,0.00,0.00,0.00,0.02,0.00,0.00,0.04,0.00,0.00,0.00
4,Chancery Lane,0.00,0.02,0.00,0.00,0.00,0.02,0.00,0.00,0.00,...,0.00,0.00,0.00,0.04,0.00,0.02,0.02,0.00,0.02,0.00
5,Charing Cross,0.00,0.00,0.04,0.04,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.04,0.00,0.00,0.00
6,Covent Garden,0.00,0.00,0.00,0.00,0.02,0.00,0.00,0.00,0.06,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
7,Edgware Road,0.00,0.00,0.02,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.02
8,Embankment,0.00,0.00,0.04,0.02,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.04,0.00,0.00,0.00
9,Great Portland Street,0.02,0.00,0.02,0.00,0.00,0.02,0.02,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.02,0.00,0.00,0.00


In [39]:
# Size of dataframe
df_london_grouped.shape

(28, 199)

In [107]:
# Examine top 5 venue categories for each Tube station
num_top_venues = 5

for hood in df_london_grouped['Station']:
    print("----"+hood+"----")
    temp = df_london_grouped[df_london_grouped['Station'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Baker Street----
               venue  freq
0             Garden  0.06
1              Hotel  0.06
2  French Restaurant  0.04
3        Coffee Shop  0.04
4       Burger Joint  0.04


----Blackfriars----
                venue  freq
0         Coffee Shop  0.10
1                 Pub  0.08
2          Art Museum  0.06
3  Falafel Restaurant  0.06
4      Scenic Lookout  0.06


----Bond Street----
               venue  freq
0              Hotel  0.08
1  French Restaurant  0.06
2        Art Gallery  0.06
3     Clothing Store  0.06
4             Lounge  0.04


----Borough----
                venue  freq
0         Coffee Shop  0.08
1  Italian Restaurant  0.06
2  Seafood Restaurant  0.06
3                 Pub  0.06
4                Café  0.04


----Chancery Lane----
                   venue  freq
0            Coffee Shop  0.16
1                    Pub  0.10
2     Falafel Restaurant  0.06
3                  Hotel  0.04
4  Vietnamese Restaurant  0.04


----Charing Cross----
            venue  freq

In [40]:
# Define a function to sort the venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [41]:
# Create a new dataframe and display the top 10 venues for each Tube station
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Station']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
df_london_venues_sorted = pd.DataFrame(columns=columns)
df_london_venues_sorted['Station'] = df_london_grouped['Station']

for ind in np.arange(df_london_grouped.shape[0]):
    df_london_venues_sorted.iloc[ind, 1:] = return_most_common_venues(df_london_grouped.iloc[ind, :], num_top_venues)

df_london_venues_sorted.head()

,Station,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Baker Street,Hotel,Garden,Sandwich Place,French Restaurant,Museum,Burger Joint,Movie Theater,Coffee Shop,Restaurant,Yoga Studio
1,Blackfriars,Coffee Shop,Pub,Hotel,Scenic Lookout,Art Museum,Falafel Restaurant,Cocktail Bar,Gym / Fitness Center,Grocery Store,Pedestrian Plaza
2,Bond Street,Hotel,Clothing Store,Art Gallery,French Restaurant,Deli / Bodega,Cocktail Bar,Juice Bar,Coffee Shop,Lounge,Cosmetics Shop
3,Borough,Coffee Shop,Pub,Seafood Restaurant,Italian Restaurant,Hotel,Street Food Gathering,Restaurant,Café,Wine Bar,Food Stand
4,Chancery Lane,Coffee Shop,Pub,Falafel Restaurant,Hotel,Vietnamese Restaurant,Gym / Fitness Center,Tea Room,Beer Bar,Fast Food Restaurant,Hotel Bar


##### 4.12. Cluster neighbourhoods of Tube stations using k-means clustering

In [42]:
# set number of clusters
kclusters = 5

df_london_grouped_clustering = df_london_grouped.drop('Station', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(df_london_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 3, 0, 3, 1, 2, 3, 1, 2, 1], dtype=int32)

In [44]:
# Create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood
# add clustering labels
df_london_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

df_london_merged = df_highest_rated

# merge datasets to add latitude/longitude for each neighborhood
df_london_merged = df_london_merged.join(df_london_venues_sorted.set_index('Station'), on='Station')

df_london_merged.head()

,Station,Latitude,Longitude,Zone,Total Tip Count,Average Rating,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Baker Street,51.52265,-0.157040,1,166,7.425000,1,Hotel,Garden,Sandwich Place,French Restaurant,Museum,Burger Joint,Movie Theater,Coffee Shop,Restaurant,Yoga Studio
1,Blackfriars,51.51144,-0.104191,1,144,7.400000,3,Coffee Shop,Pub,Hotel,Scenic Lookout,Art Museum,Falafel Restaurant,Cocktail Bar,Gym / Fitness Center,Grocery Store,Pedestrian Plaza
2,Bond Street,51.51461,-0.148970,1,132,7.540000,0,Hotel,Clothing Store,Art Gallery,French Restaurant,Deli / Bodega,Cocktail Bar,Juice Bar,Coffee Shop,Lounge,Cosmetics Shop
3,Borough,51.50095,-0.094460,1,22,7.700000,3,Coffee Shop,Pub,Seafood Restaurant,Italian Restaurant,Hotel,Street Food Gathering,Restaurant,Café,Wine Bar,Food Stand
4,Chancery Lane,51.51836,-0.111150,1,92,7.766667,1,Coffee Shop,Pub,Falafel Restaurant,Hotel,Vietnamese Restaurant,Gym / Fitness Center,Tea Room,Beer Bar,Fast Food Restaurant,Hotel Bar


In [45]:
# Export popular venue categories for Zone 1 Tube station areas to CSV for use without having to regenerate all the data
df_london_merged.to_csv("london_merged.csv", encoding='utf-8', index=False)

In [47]:
# Debug: popular venue categories for Zone 1 Tube station areas from CSV instead of regenerating all the data
df_london_merged = pd.read_csv("london_merged.csv")

In [48]:
# Visualise clusters on a map of London
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_london_merged['Latitude'], 
                                  df_london_merged['Longitude'], 
                                  df_london_merged['Station'], 
                                  df_london_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [58]:
# Examine Cluster 1
df_london_merged.loc[df_london_merged['Cluster Labels'] == 0, df_london_merged.columns[[0] + list(range(1, df_london_merged.shape[1]))]]

,Station,Latitude,Longitude,Zone,Total Tip Count,Average Rating,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Bond Street,51.514610,-0.148970,1,132,7.54,0,Hotel,Clothing Store,Art Gallery,French Restaurant,Deli / Bodega,Cocktail Bar,Juice Bar,Coffee Shop,Lounge,Cosmetics Shop
11,High Street Kensington,51.500346,-0.192352,1,25,8.10,0,Café,Restaurant,Hotel,French Restaurant,Bakery,Clothing Store,Juice Bar,Gym / Fitness Center,Burger Joint,Pub
20,Oxford Circus,51.515170,-0.141190,1,233,8.10,0,Coffee Shop,Clothing Store,Cocktail Bar,Cosmetics Shop,Pizza Place,Hotel,English Restaurant,Art Gallery,Indian Restaurant,Department Store
24,Sloane Square,51.492585,-0.156091,1,90,7.50,0,Bakery,Plaza,French Restaurant,Hotel,Boutique,Clothing Store,Indian Restaurant,Italian Restaurant,Deli / Bodega,Gastropub


Analysis: Cluster 1 appears to have a range of venues featuring shops, different continental restaurants and hotels which may also have restaurants.

In [49]:
# Rename Cluster 1
df_london_merged.loc[df_london_merged['Cluster Labels'] == 0, ['Cluster Labels']] = 'Shopping & Restaurants' 


In [59]:
# Examine Cluster 2
df_london_merged.loc[df_london_merged['Cluster Labels'] == 1, df_london_merged.columns[[0] + list(range(1, df_london_merged.shape[1]))]]

,Station,Latitude,Longitude,Zone,Total Tip Count,Average Rating,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Baker Street,51.522650,-0.157040,1,166,7.425000,1,Hotel,Garden,Sandwich Place,French Restaurant,Museum,Burger Joint,Movie Theater,Coffee Shop,Restaurant,Yoga Studio
4,Chancery Lane,51.518360,-0.111150,1,92,7.766667,1,Coffee Shop,Pub,Falafel Restaurant,Hotel,Vietnamese Restaurant,Gym / Fitness Center,Tea Room,Beer Bar,Fast Food Restaurant,Hotel Bar
7,Edgware Road,51.520170,-0.170226,1,16,7.200000,1,Middle Eastern Restaurant,Café,Lebanese Restaurant,Pub,Coffee Shop,Japanese Restaurant,Beer Bar,Persian Restaurant,Grocery Store,Hotel
9,Great Portland Street,51.523910,-0.143970,1,112,7.120000,1,Coffee Shop,Pizza Place,Café,Garden,Middle Eastern Restaurant,Pub,American Restaurant,Burger Joint,Seafood Restaurant,Science Museum
17,Marylebone,51.522660,-0.162996,1,68,7.033333,1,Pub,Hotel,Gastropub,Café,Museum,Movie Theater,Pizza Place,Coffee Shop,Restaurant,Sandwich Place
22,Queensway,51.510484,-0.187050,1,6,8.500000,1,Pub,Hotel,Chinese Restaurant,Coffee Shop,Garden,Greek Restaurant,Bakery,Gym / Fitness Center,Health & Beauty Service,Palace
23,Regent's Park,51.523440,-0.147130,1,93,7.000000,1,Garden,Hotel,Coffee Shop,Gastropub,French Restaurant,Middle Eastern Restaurant,Café,Diner,Restaurant,Pub


Analysis: Cluster 2 has an abundance of eating and drinking establishments.

In [50]:
# Rename Cluster 2
df_london_merged.loc[df_london_merged['Cluster Labels'] == 1, ['Cluster Labels']] = 'Food & Drink' 


In [62]:
# Examine Cluster 3
df_london_merged.loc[df_london_merged['Cluster Labels'] == 2, df_london_merged.columns[[0] + list(range(1, df_london_merged.shape[1]))]]

,Station,Latitude,Longitude,Zone,Total Tip Count,Average Rating,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Charing Cross,51.507108,-0.122963,1,465,8.10,2,Theater,Ice Cream Shop,Hotel,Plaza,Wine Bar,Art Gallery,Art Museum,Garden,Cocktail Bar,Spanish Restaurant
8,Embankment,51.507170,-0.121950,1,453,7.88,2,Theater,Spanish Restaurant,Wine Bar,Art Gallery,Hotel,Plaza,Garden,Cocktail Bar,Park,Spa
14,Leicester Square,51.511480,-0.128490,1,513,7.62,2,Theater,Ice Cream Shop,Coffee Shop,Bakery,Steakhouse,Japanese Curry Restaurant,Lounge,General Entertainment,Speakeasy,Spanish Restaurant
21,Piccadilly Circus,51.510220,-0.133920,1,244,7.84,2,Hotel,Theater,Steakhouse,Clothing Store,Bookstore,Gourmet Shop,Ice Cream Shop,Dessert Shop,Gift Shop,Speakeasy
26,Temple,51.511140,-0.113410,1,147,7.46,2,Theater,Hotel,Cocktail Bar,Art Gallery,Pub,Coffee Shop,Restaurant,Scenic Lookout,Event Space,Falafel Restaurant
27,Tottenham Court Road,51.516400,-0.130270,1,313,7.26,2,Theater,Hotel,Coffee Shop,Chocolate Shop,Steakhouse,Bookstore,Exhibit,Pizza Place,Lounge,Beer Bar


Analysis: Cluster 3 is predominantly the theatre and hotel district.

In [51]:
# Rename Cluster 3
df_london_merged.loc[df_london_merged['Cluster Labels'] == 2, ['Cluster Labels']] = 'Theatres' 


In [64]:
# Examine Cluster 4
df_london_merged.loc[df_london_merged['Cluster Labels'] == 3, df_london_merged.columns[[0] + list(range(1, df_london_merged.shape[1]))]]

,Station,Latitude,Longitude,Zone,Total Tip Count,Average Rating,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Blackfriars,51.511440,-0.104191,1,144,7.400000,3,Coffee Shop,Pub,Hotel,Scenic Lookout,Art Museum,Falafel Restaurant,Cocktail Bar,Gym / Fitness Center,Grocery Store,Pedestrian Plaza
3,Borough,51.500950,-0.094460,1,22,7.700000,3,Coffee Shop,Pub,Seafood Restaurant,Italian Restaurant,Hotel,Street Food Gathering,Restaurant,Café,Wine Bar,Food Stand
6,Covent Garden,51.513080,-0.124270,1,279,7.400000,3,Ice Cream Shop,Dessert Shop,Bakery,Coffee Shop,Dance Studio,Theater,Sushi Restaurant,Pizza Place,Steakhouse,Seafood Restaurant
15,London Bridge,51.505353,-0.084826,1,71,7.466667,3,Hotel,Coffee Shop,Scenic Lookout,Restaurant,Wine Bar,Italian Restaurant,Seafood Restaurant,Cocktail Bar,Food Truck,Cheese Shop
16,Mansion House,51.512560,-0.093970,1,75,7.240000,3,Coffee Shop,Italian Restaurant,Café,Gym / Fitness Center,Roof Deck,Cocktail Bar,Steakhouse,Vietnamese Restaurant,Hotel,Wine Bar
18,Monument,51.510165,-0.085991,1,42,7.325000,3,Hotel,Coffee Shop,Gym / Fitness Center,Historic Site,Garden,French Restaurant,Steakhouse,Scenic Lookout,Cocktail Bar,Salad Place
19,Old Street,51.526180,-0.087510,1,188,7.175000,3,Coffee Shop,Café,Cocktail Bar,Hotel,Pub,Steakhouse,Burger Joint,Japanese Restaurant,Gym / Fitness Center,Food Truck
25,Southwark,51.503840,-0.104780,1,108,7.666667,3,Hotel,Cocktail Bar,Gym / Fitness Center,Bar,Café,Art Museum,Coffee Shop,Gastropub,Burger Joint,Theater


Analysis: Cluster 4 has a lot of coffee shops and fitness centres suggesting it may be providing services to businesses. There are also a range of restaurants.

In [52]:
# Rename Cluster 4
df_london_merged.loc[df_london_merged['Cluster Labels'] == 3, ['Cluster Labels']] = 'Business & Restaurants' 


In [66]:
# Examine Cluster 5
df_london_merged.loc[df_london_merged['Cluster Labels'] == 4, df_london_merged.columns[[0] + list(range(1, df_london_merged.shape[1]))]]

,Station,Latitude,Longitude,Zone,Total Tip Count,Average Rating,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,Green Park,51.50674,-0.14276,1,138,7.80,4,Boutique,Dessert Shop,Hotel,Lounge,Hotel Bar,Seafood Restaurant,Plaza,Indian Restaurant,Tea Room,Clothing Store
12,Hyde Park Corner,51.50313,-0.15278,1,165,8.25,4,Hotel,Café,Plaza,Tea Room,French Restaurant,Restaurant,American Restaurant,Pakistani Restaurant,Chinese Restaurant,Garden
13,Knightsbridge,51.50169,-0.16030,1,813,8.66,4,Boutique,Hotel,Italian Restaurant,Café,Champagne Bar,Shoe Store,Clothing Store,Coffee Shop,Japanese Restaurant,Department Store


Analysis: Cluster 5 has more of an upmarket feel to it with boutique shopping and fancy restaurants.

In [53]:
# Rename Cluster 5
df_london_merged.loc[df_london_merged['Cluster Labels'] == 4, ['Cluster Labels']] = 'Upmarket & Restaurants' 


In [68]:
# Create findings dataframe and rename cluster label column
df_london_merged.rename(columns = {'Cluster Labels':'Cluster'}, inplace = True)
df_london_findings = df_london_merged
df_london_findings.head()

,Station,Latitude,Longitude,Zone,Total Tip Count,Average Rating,Cluster,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Baker Street,51.52265,-0.157040,1,166,7.425000,Food & Drink,Hotel,Garden,Sandwich Place,French Restaurant,Museum,Burger Joint,Movie Theater,Coffee Shop,Restaurant,Yoga Studio
1,Blackfriars,51.51144,-0.104191,1,144,7.400000,Business & Restaurants,Coffee Shop,Pub,Hotel,Scenic Lookout,Art Museum,Falafel Restaurant,Cocktail Bar,Gym / Fitness Center,Grocery Store,Pedestrian Plaza
2,Bond Street,51.51461,-0.148970,1,132,7.540000,Shopping & Restaurants,Hotel,Clothing Store,Art Gallery,French Restaurant,Deli / Bodega,Cocktail Bar,Juice Bar,Coffee Shop,Lounge,Cosmetics Shop
3,Borough,51.50095,-0.094460,1,22,7.700000,Business & Restaurants,Coffee Shop,Pub,Seafood Restaurant,Italian Restaurant,Hotel,Street Food Gathering,Restaurant,Café,Wine Bar,Food Stand
4,Chancery Lane,51.51836,-0.111150,1,92,7.766667,Food & Drink,Coffee Shop,Pub,Falafel Restaurant,Hotel,Vietnamese Restaurant,Gym / Fitness Center,Tea Room,Beer Bar,Fast Food Restaurant,Hotel Bar


In [82]:
# Export London findings to CSV for use without having to regenerate all the data
df_london_findings.to_csv("london_findings.csv", encoding='utf-8', index=False)

In [54]:
# Debug: import London findings from CSV instead of regenerating all the data
df_london_findings = pd.read_csv("london_findings.csv")

##### 4.13. Combine findings to visualise highly rated bars alongside clusters of nearby venue categories

In [55]:
# Draw map with Tube stations in Zone 1 and Average Rating
map_london_findings = folium.Map(location=[latitude, longitude], tiles='Stamen Toner', zoom_start=13)
for lat, lng, station, tip, rating, cluster in zip(df_london_findings['Latitude'], 
                                   df_london_findings['Longitude'], 
                                   df_london_findings['Station'],
                                   df_london_findings['Total Tip Count'],
                                   df_london_findings['Average Rating'],
                                   df_london_findings['Cluster']):
    popup = folium.Popup(station + ": Average Rating " + str(round(rating,1)) + ", Total Tip Count " + str(tip), parse_html=True)
    cluster_popup = folium.Popup(station + ": Cluster " + cluster, parse_html=True)
    if (rating <= 6):
        marker_color = 'red',
        fill_color = '#F08080'
    elif (rating > 6 and rating <= 7):
        marker_color = 'orange',
        fill_color = '#FFE4B5'
    elif (rating > 7 and rating <= 8):
        marker_color = 'yellow',
        fill_color = '#FFFFE0'
    elif (rating > 8 and rating <= 9):
        marker_color = 'green',
        fill_color = '#98FB98'
    elif (rating > 9 and rating <= 10):
        marker_color = 'blue',
        fill_color = '#3186cc'
    else:
        color = 'black'
    if (cluster == 'Shopping & Restaurants'):
        cluster_color = '#1B4F72'
    elif (cluster == 'Food & Drink'):
        cluster_color = '#2874A6'
    elif (cluster == 'Theatres'):
        cluster_color = '#9B59B6'
    elif (cluster == 'Business & Restaurants'):
        cluster_color = '#4A235A'
    elif (cluster == 'Upmarket & Restaurants'):
        cluster_color = '#AED6F1'
    else:
        cluster_color = 'black'
    folium.Circle(
        [lat, lng],
        radius = 1000,
        popup = cluster_popup,
        color = cluster_color,
        parse_html = False).add_to(map_london_findings)
    folium.CircleMarker(
        [lat, lng],
        radius = 5,
        popup = popup,
        color=marker_color,
        fill = True,
        fill_color = fill_color,
        fill_opacity = 0.7,
        parse_html = False).add_to(map_london_findings)  
    
map_london_findings

##### 5. Results Discussion

##### 5.1. Which Tube stations are nearest the highest rated bars and pubs on Foursquare?

The top 6 rated Tube stations areas in Zone 1 of London, all having an average rating of over 8, are:

Knightsbridge - 8.7

Queensway - 8.5

Hyde Park Corner - 8.2

Charing Cross - 8.1

Oxford Circus - 8.1

High Street Kensington - 8.1


##### 5.2. What are 10 most common venue categories nearby?

In [106]:
df_london_findings[df_london_findings['Average Rating'] > 8]

,Station,Latitude,Longitude,Zone,Total Tip Count,Average Rating,Cluster,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Charing Cross,51.507108,-0.122963,1,465,8.10,Theatres,Theater,Ice Cream Shop,Hotel,Plaza,Wine Bar,Art Gallery,Art Museum,Garden,Cocktail Bar,Spanish Restaurant
11,High Street Kensington,51.500346,-0.192352,1,25,8.10,Shopping & Restaurants,Café,Restaurant,Hotel,French Restaurant,Bakery,Clothing Store,Juice Bar,Gym / Fitness Center,Burger Joint,Pub
12,Hyde Park Corner,51.503130,-0.152780,1,165,8.25,Upmarket & Restaurants,Hotel,Café,Plaza,Tea Room,French Restaurant,Restaurant,American Restaurant,Pakistani Restaurant,Chinese Restaurant,Garden
13,Knightsbridge,51.501690,-0.160300,1,813,8.66,Upmarket & Restaurants,Boutique,Hotel,Italian Restaurant,Café,Champagne Bar,Shoe Store,Clothing Store,Coffee Shop,Japanese Restaurant,Department Store
20,Oxford Circus,51.515170,-0.141190,1,233,8.10,Shopping & Restaurants,Coffee Shop,Clothing Store,Cocktail Bar,Cosmetics Shop,Pizza Place,Hotel,English Restaurant,Art Gallery,Indian Restaurant,Department Store
22,Queensway,51.510484,-0.187050,1,6,8.50,Food & Drink,Pub,Hotel,Chinese Restaurant,Coffee Shop,Garden,Greek Restaurant,Bakery,Gym / Fitness Center,Health & Beauty Service,Palace


##### 5.3. Which group of Tube stations offer the best choice for a meal after drinks?

The overlay of clusters and the Tube stations which have bars which have an average rating of over 7, shows blue circles for restaurant related clusters, light purple for theatre related and dark purple for business related. The green coloured Tube stations, having bars rated over 8, around Hyde Park fall neatly into restaurant areas but the fact that they surround the park means that moving between areas to explore other areas will involve walking through sparsely populated areas. 

##### 5.5. Limitations

The restrictions on calls to the Foursquare API meant that the data was focussed on just Zone 1 and only examining 5 bars per Tube station area. Without the restrictions it would be possible to look at the whole city and all of the bars in each area. This will produce different results. 

The clusters determined by the k-means algorithm are still very broad and do not necessarily produce clusters of Tube stations that are geographically near each other. For example Sloane Square is separated from Oxford Circus and nearby Stations by parks, despite being a similar type of area in terms of categorising venues.

Where Tube stations are very close together, there may be an element of double counting the same venues, giving an impression that there may be more choice than actually true.

Eliminating further exploration of Tube station areas where the bars have an average rating of less than 7, might miss some good nearby areas for food which might co-incidentally be not that far away from a different Tube station with a higher rating.

The average rating of the bars doesn't take into account the number of ratings (just that there are one or more ratings). With more time this could be refined. 

##### Conclusions

Which Tube station should we meet at for a successful night out?


On balance, Oxford Circus has highly rated bars (with average rating of over 8), is in a restaurant area and is surrounded by lots of other close by Tube stations (with bars with an average rating of over 7) which themselves are in restaurant areas. It borders the Theatre district, which is not devoid of restaurants either, and may offer a good atmosphere for the evening. Based on this preliminary analysis, our group would meet at Oxford Circus to start our evening out.